In [ ]:
import os
import numpy as np
import pandas as pd
from keras.utils import np_utils, img_to_array, load_img
from sklearn.preprocessing import LabelEncoder

from keras.applications.resnet import preprocess_input
from keras.applications import imagenet_utils

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras.callbacks import EarlyStopping

In [ ]:
base_dir = '.'
data_dir = os.path.join(base_dir,'train')
files = os.listdir(data_dir)

In [ ]:
labels = pd.read_csv(os.path.join(base_dir,'labels.csv'))
labels.head()

In [ ]:
file_df = pd.DataFrame({'id':list(map(lambda x:x.replace('.jpg',''),files))})
file_df.head()

In [ ]:
label_info = pd.merge(left=file_df, right=labels)
label_info.head()

In [ ]:
img = plt.imread(os.path.join(data_dir,files[1]))

In [ ]:
plt.imshow(img)
plt.title(label_info.iloc[1]['breed'])
plt.show()

Converting target to One-Hot-Vector Format

In [ ]:
num_classes = len(label_info.breed.unique())
num_classes

In [ ]:
le = LabelEncoder()
breed = le.fit_transform(label_info.breed)
Y = np_utils.to_categorical(breed, num_classes=num_classes)

In [ ]:
Y.shape

Converting images to numpy array

In [ ]:
input_dim = (224,224)

X = np.zeros((Y.shape[0],*input_dim,3))

for i, img in enumerate(files):
    image = load_img(os.path.join(data_dir,img),target_size=input_dim)
    image = img_to_array(image)
    image = image.reshape((1,*image.shape))
    image = preprocess_input(image)
    X[i] = image

In [ ]:
X.shape

Create Callbacks

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')

Triaining only last layer

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Flatten, Dropout

vgg_model = VGG19(weights='imagenet', include_top=False)

x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
out = Dense(120, activation='softmax')(x)

model_1 = Model(inputs=vgg_model.input, outputs=out)

for layer in vgg_model.layers:
    layer.trainable = False

from keras.optimizers import Adam
opt = Adam()

model_1.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model_1.summary()

In [ ]:
history_last_layer = model_1.fit(X,Y,batch_size=256,epochs=20,validation_split=0.2, verbose=2,callbacks=[earlystop])
model_1.save('model_1.hs')

In [ ]:
history_last_layer.history.keys()

In [ ]:
val_acc = history_last_layer.history.get('val_acc')

In [ ]:
acc = history_last_layer.history.get('acc')

In [ ]:
overfit_info = pd.DataFrame({'acc':acc, 'val_acc':val_acc})

In [ ]:
overfit_info.plot.line()

predictions

In [ ]:
one_layer_model = load_model('model_1.h5')

In [ ]:
image = load_img('mydog.jpg', target_size=input_dim)
image = img_to_array(image)
image = image.reshape((1, *image.shape))
image = preprocess_input(image)

In [ ]:
res = one_layer_model.predict(image)

In [ ]:
s = np.argsort(res)[0][-5:]

In [ ]:
le.inverse_transform(s)

Train More

In [ ]:
x = vgg_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
out = Dense(120, activation='softmax')(x)

model_2 = Model(inputs=vgg_model.input, outputs=out)

for layer in vgg_model.layers[:-1]:
    layer.trainable=False

for layer in vgg_model.layers[-1:]:
    layer.trainable=True

model_2.compile(optimizer=opt, loss='categorical_crossentropy',
              metrics=['accuracy'])
model_2.summary()


In [ ]:
history_more_layers = model_2.fit(X,Y,batch_size=256,epochs=20,validation_split=0.2, verbose=2,callbacks=[earlystop])
model_2.save('model_2.hs')

In [ ]:
new_model = load_model('model_2.h5')

In [ ]:
res_2 = new_model.predict(image)

In [ ]:
s = np.argsort(res_2)[0][-5:]

In [ ]:
le.inverse_transform(s)

Image Augumentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
datagen.fit(X)

In [ ]:
model=load_model('model_2.h5')

In [ ]:
history_aug_images = model.fit_generator(datagen.flow(X, Y, batch_size=256), steps_per_epoch=len(X)/32, verbose=2)

In [ ]:
history_aug_images.history.keys()

In [ ]:
model.save('model_3.h5')

In [ ]:
aug_model = load_model('model_3.h5')

In [ ]:
res_3 = aug_model.predict(image)

In [ ]:
s = np.argsort(res_3)[0][-5:]

In [ ]:
le.inverse_transform(s)
